# load library

In [ ]:
# auto reloading of local scripts under dev
%load_ext autoreload
%autoreload 2

In [ ]:
# relying on these stdlib anyway
import http
import re
import os
import sys
import datetime

from dateutil import parser
import pandas as pd

In [ ]:
# Google Cloud and FISS
from firecloud import api as fapi

from google.cloud import storage
storage_client = storage.Client()

In [ ]:
# load local lib
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.gcs_utils import GcsPath

from src.terra.table_utils import *
from src.terra.submission.submission_utils import verify_before_submit

# load data

In [ ]:
primary_namespace = 'production-long-reads'
primary_workspace = 'broad-gp-pacbio'
root_data_type = 'sample'

In [ ]:
# primary_namespace = 'broad-firecloud-dsde-methods'
# primary_workspace = 'HGSVC2-unified'
# root_data_type = 'ccs-flowcell'

In [ ]:
flowcell_table = \
  fetch_existing_root_table(ns=primary_namespace,
                            ws=primary_workspace,
                            etype=root_data_type)

In [ ]:
gcs_locations = ['aligned_bai', 'aligned_bam', 'aligned_pbi',
                 'ccs_bam', 'ccs_pbi', 'ccs_report',
                 'fingerprint_details', 'fingerprint_metrics',
                 'fq', 'gcs_input_dir', 'input_bam', 'input_pbi', 'subreads_bam', 'subreads_pbi']

In [ ]:
lab_identity = ['bio_sample', 'description', 'well_sample']
sequencer_identity = ['flowcell_id', 'movie_name', 'well_name']
terra_identity = ['sample']

In [ ]:
categorical_columns = {'type': 'category',
                       'columns': ['application', 'experiment_type', 'instrument', 'workspace']}

date_time_columns = {'type': 'datetime64',
                     'timezone': datetime.timezone.utc,
                     'columns': ['created_at']}

boolean_columns = {'type': 'bool',
                   'columns': ['is_ccs', 'is_corrected', 'is_isoseq']}

int_type_columns = {'type': 'Int64',
                    'columns': ['aligned_num_bases','aligned_num_reads','aligned_read_length_N50',
                                'ccs_zmws_fail_filters','ccs_zmws_input','ccs_zmws_pass_filters', 'ccs_zmws_shortcut_filters',
                                'insert_size',
                                'num_bases','num_reads','num_reads_Q10','num_reads_Q12','num_reads_Q15','num_reads_Q5','num_reads_Q7','num_records',
                                'total_length']}

float_type_columns = {'type': 'float64',
                      'columns': ['lod_expected_sample',
                                  'aligned_est_fold_cov', 'raw_est_fold_cov',
                                  'aligned_frac_bases','aligned_read_length_mean','aligned_read_length_median','aligned_read_length_stdev',
                                  'average_identity', 'median_identity',
                                  'ccs_zmws_fail_filters_pct','ccs_zmws_pass_filters_pct','ccs_zmws_shortcut_filters_pct',
                                  'polymerase_read_length_N50', 'polymerase_read_length_mean',
                                  'read_length_N50', 'read_length_mean', 'read_length_median', 'read_length_stdev', 'read_qual_mean', 'read_qual_median',
                                  'subread_read_length_N50','subread_read_length_mean']}

string_type_columns = {'type': 'str',
                       'columns': gcs_locations + terra_identity + lab_identity + sequencer_identity}

In [ ]:
for n in boolean_columns['columns']:
    flowcell_table[n] = flowcell_table[n].apply(lambda s: s=='TRUE' or s=='True' or s=='true').astype(boolean_columns['type'])

In [ ]:
for n in categorical_columns['columns']:
    flowcell_table[n] = flowcell_table[n].astype(categorical_columns['type'])

In [ ]:
for n in string_type_columns['columns']:
    flowcell_table[n] = flowcell_table[n].astype(string_type_columns['type'])

In [ ]:
def convert_to_float(e) -> float or None:
    if e:
        if e.lower() in ['nan', 'none']:
            return None
        else:
            try:
                return float(e)
            except TypeError:
                print(e)
                raise
    else:
        return None

def convert_to_int(e) -> int:
    f = convert_to_float(e)
    return round(f) if f else None

In [ ]:
for n in int_type_columns['columns']:
    try:
        flowcell_table[n] = flowcell_table[n].apply(convert_to_int).astype(int_type_columns['type'])
    except ValueError:
        print(n)
        raise

In [ ]:
for n in float_type_columns['columns']:
    try:
        flowcell_table[n] = flowcell_table[n].apply(convert_to_float).astype(float_type_columns['type'])
    except ValueError:
        print(n)
        raise

In [ ]:
def convert_date_time(s):
    try:
        t = parser.isoparse(s).astimezone(tz=date_time_columns['timezone'])
        return pd.to_datetime(t)
    except (ValueError, pd.errors.OutOfBoundsDatetime):
        return pd.Timestamp.min
for n in date_time_columns['columns']:
    flowcell_table[n] = flowcell_table[n].apply(lambda s: pd.to_datetime(convert_date_time(s)))

# PLAY

In [ ]:
verify_before_submit(hgsvc2_namespace, hgsvc2_workspace,
                     workflow_name='Dummy',
                     etype='ccs-flowcell', enames=hgsvc2_ccs_flowcells['ccs-flowcell'][0:4].tolist(),
                     expression='this.ccs-flowcells',
                     use_callcache=True)